# Step.2 Data Cleaning and Manipulation:
Data cleaning was performed to identify and address errors, handle missing values, and ensure consistency across the dataset, preparing it for further analysis

**2.1 Import neccesary libraries**

In [3]:
import pandas as pd
#for checking misspells
from fuzzywuzzy import fuzz
import unidecode

C:\Users\ASUS\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


**2.2 Read the datasets**

In [5]:
# Read files into DataFrames

df_geolocation = pd.read_csv("olist_geolocation_dataset_2.0.csv") #Use the processed dataset: olist_geolocation_dataset_2.0.
df_orders = pd.read_csv("olist_orders_dataset.csv")
df_payments = pd.read_csv("olist_order_payments_dataset.csv")
df_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
df_products = pd.read_csv("olist_products_dataset.csv")
df_product_translation = pd.read_csv("product_category_name_translation.csv")
df_customers = pd.read_csv("olist_customers_dataset.csv")
df_sellers = pd.read_csv("olist_sellers_dataset.csv")
df_items = pd.read_csv("olist_order_items_dataset.csv")

**2.3 Process all datasets**

**2.3.1 Orders Dataset**

In [8]:
#%%# List of date columns
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

# Check the NA values and duplicates
df_orders.isna().sum()
df_orders['order_id'].duplicated().sum()

# Extracts just the date part
df_orders[date_columns] = df_orders[date_columns].apply(pd.to_datetime, errors='coerce')
df_orders = df_orders.dropna(subset=date_columns)

# Check the integrity of the date columns based on the new conditions
integrity_check = (
    (df_orders['order_purchase_timestamp'] <= df_orders['order_approved_at']) &  # order_purchase_timestamp should be earlier than order_approved_at
    (df_orders['order_approved_at'] <= df_orders['order_delivered_carrier_date']) &  # order_approved_at should be earlier than order_delivered_carrier_date
    (df_orders['order_delivered_carrier_date'] <= df_orders['order_delivered_customer_date'])  # order_delivered_carrier_date should be earlier than order_delivered_customer_date
)

# Remove rows that violate the integrity checks
df_orders = df_orders[integrity_check]

**2.3.2 Payments Dataset**

In [10]:
# Checking for NA values: no nan value
df_payments.isna().sum()

# Checking for duplicates: 4446 duplicates
df_payments['order_id'].duplicated().sum()

# Displaying duplicate rows:
duplicates = df_payments[df_payments.duplicated(subset='order_id', keep=False)].sort_values(by='order_id')
#The duplicate order_id in the payments are due to multiple payment methods for one order, such as by credit card and by voucher, which would result in two rows.

# Select only column 'order_id' , 'payment_value' , 'payment_type'
df_payments = df_payments[['order_id','payment_type', 'payment_value']]

# Based on the above discovery, combining the same orders into the same rows:
df_payments = df_payments.groupby('order_id').agg(
    payment_value_sum=('payment_value', 'sum'),
    payment_types_combined=('payment_type', lambda x: ','.join(sorted(set(x))))
).reset_index()

**2.3.3 Reviews Dataset**

In [12]:
# Select only column 'review_id', 'order_id' , 'review_score' , 'review_comment_message'
df_reviews['is_review'] = df_reviews['review_comment_message'].apply(lambda x: 1 if pd.notna(x) else 0)
df_reviews = df_reviews[['review_id', 'order_id', 'review_score','is_review']]

# Some review_id may correspond to multiple orders.
duplicates_reviews = df_reviews[df_reviews['review_id'].duplicated(keep=False)]

**2.3.4 Products Dataset**

In [14]:
# Merge translation datasets to translate product categories into English.
df_products = df_products.merge(df_product_translation, on='product_category_name', how='left')

# Select only column 'product_id', 'product_category_name' , 'product_description_lenght' , 'product_photos_qty','product_category_name_english'
df_products = df_products[['product_id', 'product_category_name','product_photos_qty','product_category_name_english']]

#Drop all null values
df_products.isnull().sum()
df_products = df_products.dropna()

**2.3.5 Customers Dataset**

In [16]:
# Check N/A & duplicated value
df_customers.isnull().sum()
df_customers['customer_id'].duplicated().sum()
df_customers['customer_unique_id'].duplicated().sum() # that is acceptable as each of the customer may make several orders(customer_id in this dataset)

# Joined with geolocation dataset to know each customer latitude and longitude.
df_customers = df_customers.merge(df_geolocation,
                                  left_on='customer_zip_code_prefix',right_on = 'geolocation_zip_code_prefix',how = 'left')
df_customers  = df_customers.drop(columns=['geolocation_city','normalized_city'])

df_customers['is_same_state'] = df_customers['customer_state'] == df_customers['geolocation_state']

# Check that the geographic information in the customer dataset is consistent with the geolocation data set.
df_customers = df_customers[df_customers['is_same_state']!=False]

# Change the name of the columns to avoid confusing columns after combination of data
df_customers = df_customers.rename(columns={'geolocation_lat': 'customer_lat_n', 'geolocation_lng': 'customer_lng_n'})
df_customers = df_customers.rename(columns={'geolocation_state': 'customer_state_n', 'corrected_city': 'customer_city_n'})

# Select only column 'customer_id', 'customer_unique_id' , 'customer_zip_code_prefix' , 'customer_state_n','customer_city_n','customer_lat_n','customer_lng_n'
df_customers = df_customers[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix','customer_state_n','customer_city_n','customer_lat_n','customer_lng_n']]

**2.3.6 Sellers Dataset**

In [18]:
df_sellers.isnull().sum()
df_sellers['seller_id'].duplicated().sum()

## Joined with geolocation dataset to know each seller latitude and longitude.
df_sellers = df_sellers.merge(df_geolocation,left_on='seller_zip_code_prefix',right_on = 'geolocation_zip_code_prefix',how = 'left' )
df_sellers  = df_sellers.drop(columns=['geolocation_city','normalized_city'])

# Check that the geographic information in the seller dataset is consistent with the geolocation data set.
df_sellers['is_same_state'] = df_sellers['seller_state'] == df_sellers['geolocation_state']
df_sellers['is_same_city'] = df_sellers['seller_city'] == df_sellers['corrected_city']
different_state_or_city = df_sellers[(df_sellers['is_same_state'] == False) | (df_sellers['is_same_city'] == False)]

df_sellers = df_sellers[(df_sellers['is_same_state'] != False) & (df_sellers['is_same_city'] != False)]
df_sellers = df_sellers.dropna()

# Change the name of the columns to avoid confusing columns after combination of data
df_sellers = df_sellers.rename(columns={'geolocation_lat': 'seller_lat_n', 'geolocation_lng': 'seller_lng_n'})
df_sellers = df_sellers.rename(columns={'geolocation_state': 'seller_state_n', 'corrected_city': 'seller_city_n'})

df_sellers = df_sellers[['seller_id', 'seller_zip_code_prefix','seller_city_n','seller_state_n','seller_lat_n','seller_lng_n']]

**2.3.7 Order Item Dataset**

In [20]:
# Find and drop duplicates in order items dataset
duplicate_ORDER_ITEMS = df_items[df_items.duplicated(subset='order_id', keep=False)]
df_items = df_items.drop_duplicates(subset='order_id', keep='first')

# Merge:
Merge the data and store in a new csv file which will be used for feature modelling.

In [22]:
#%% Merge data

df = df_reviews.merge(df_orders, on='order_id', how='left', indicator=True) # the orders without review won't included
df = df.merge(df_customers, on='customer_id', how='left')
df = df.merge(df_items, on='order_id', how='left')
df = df.merge(df_payments, on='order_id', how='left')
df = df.merge(df_products, on='product_id', how='left')
df = df.merge(df_sellers, on='seller_id', how='left')

#DROP unnecessary columns
df.drop(columns=[
    'customer_id',
    '_merge',
    'customer_zip_code_prefix',
    'order_item_id',
    'product_category_name',
    'seller_zip_code_prefix'],
    inplace=True)

# Rename the columns
df = df.rename(columns={'customer_state_n': 'customer_state',
                        'customer_city_n': 'customer_city',
                        'customer_lat_n':'customer_lat',
                        'customer_lng_n':'customer_lng',
                        'payment_value_sum':'payment_value',
                        'payment_types_combined':'payment_types',
                        'product_category_name_english':'product_category_name',
                        'seller_city_n':'seller_city',
                        'seller_state_n':'seller_state',
                        'seller_lat_n':'seller_lat',
                        'seller_lng_n':'seller_lng'})


duplicates = df[df.duplicated()] #0
df_na_rows = df[df.isna().any(axis=1)]
df = df.dropna()

In [23]:
df.to_csv('datasetfinal2.0.csv', index=False)